In [1]:
import pandas as pd    
import geopandas as gpd
import datetime
from geopandas.tools import geocode
#import geoplot
from shapely.geometry import Point
import numpy as np
import scipy.stats as stats
import scipy
import shapely
from shapely import speedups
speedups.enabled
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [2]:
#Census tracts
census_url = 'https://opendata.arcgis.com/datasets/de58dc3e1efc49b782ab357e044ea20c_9.geojson'
census_bounds = gpd.read_file(census_url)

In [3]:
#Bike racks
racks_url = 'https://opendata.arcgis.com/datasets/f86c29ce743e47819e588c3d643ceb63_0.geojson'
racks_data = gpd.read_file(racks_url)

In [4]:
#Parking -- street signs indicate parking zone and date of installation
street_signs_url = 'https://gisdata.seattle.gov/server/rest/services/SDOT/SDOT_Signs/MapServer/0/query?where=1%3D1&outFields=OBJECTID,COMPKEY,COMPTYPE,CATEGORY,CATEGORYDESCR,ADDDTTM,MODDTTM,INSTDATE,SHAPE_LNG,SHAPE_LAT&outSR=4326&f=json'
street_signs = gpd.read_file(street_signs_url)

In [6]:
#Bike lanes and walkways
df_SND = gpd.read_file('Street_Network_Database_SND.geojson')

In [7]:
#Population
pop_url_2010 = 'https://gisrevprxy.seattle.gov/arcgis/rest/services/CENSUS_EXT/CENSUS_2010_BASICS/MapServer/15/query?where=1%3D1&outFields=SHAPE,GEOID10,NAME10,ACRES_TOTAL,Total_Population,OBJECTID&outSR=4326&f=json'
#pop_url_2000

In [8]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']

In [9]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = ['2015']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = ['2017']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT','geometry']]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [10]:
census_columns = ['NAME10', 'SHAPE_Area', 'geometry']
census_bounds_cleaned = census_bounds[census_columns]

In [11]:
census_bounds_cleaned.head()

,NAME10,SHAPE_Area,geometry
0,25,1.059462e+07,"POLYGON ((-122.29602 47.69023, -122.29608 47.6..."
1,26,1.339838e+07,"POLYGON ((-122.30817 47.69031, -122.30947 47.6..."
2,56,3.212601e+07,"POLYGON ((-122.39300 47.63956, -122.39421 47.6..."
3,68,7.729233e+06,"POLYGON ((-122.35070 47.63994, -122.35130 47.6..."
4,60,1.413816e+07,"POLYGON ((-122.34279 47.64320, -122.34280 47.6..."


In [12]:
lane_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
bike = df_SND[lane_columns]

In [13]:
# Creates list of years in each row
snd_years = []
for i in df_SND['SNDSEG_UPD'].values:
    snd_years.append(int(i[0:4]))
    
snd_year = snd_years
snd_year_df = pd.Series(snd_year)

In [15]:
bike['YEAR'] = snd_year_df

In [16]:
#Specify segment type to extract, for walkways this will be 5
bike = bike[bike['SEGMENT_TY']==6]
bike.head()

,SEGMENT_TY,SNDSEG_UPD,SHAPE_Leng,geometry,YEAR
3,6,2004-05-19,79.522621,"LINESTRING (-122.30780 47.61410, -122.30748 47...",2004
33,6,2004-05-19,222.880092,"LINESTRING (-122.27182 47.66128, -122.27120 47...",2004
157,6,2004-05-19,251.144111,"LINESTRING (-122.29444 47.59126, -122.29342 47...",2004
241,6,2004-05-19,66.587768,"LINESTRING (-122.35275 47.62958, -122.35275 47...",2004
266,6,2004-05-19,174.233452,"LINESTRING (-122.36442 47.62976, -122.36442 47...",2004


In [17]:
bike_cleaned = bike.drop(columns=['SNDSEG_UPD', 'SEGMENT_TY'])

In [18]:
lane_by_tract = gpd.sjoin(census_bounds_cleaned, bike, op='intersects')
lanes_years = lane_by_tract.dissolve(by=['NAME10','YEAR'])
lanes_years.drop(columns=['geometry','SHAPE_Area','index_right'])
lanes_years.tail(n=16)

geometry    SHAPE_Area  \
NAME10 YEAR                                                                    
95     2012  POLYGON ((-122.28626 47.59101, -122.28660 47.5...  2.740631e+07   
       2015  POLYGON ((-122.28626 47.59101, -122.28660 47.5...  2.740631e+07   
96     2004  POLYGON ((-122.37568 47.58615, -122.37568 47.5...  1.972780e+07   
       2011  POLYGON ((-122.37568 47.58616, -122.37568 47.5...  1.972780e+07   
       2017  POLYGON ((-122.37568 47.58615, -122.37568 47.5...  1.972780e+07   
97.01  2004  POLYGON ((-122.40112 47.58365, -122.40097 47.5...  1.929115e+07   
       2011  POLYGON ((-122.40097 47.58359, -122.40112 47.5...  1.929115e+07   
97.02  2004  POLYGON ((-122.39061 47.58123, -122.38924 47.5...  2.388513e+07   
       2015  POLYGON ((-122.38924 47.58122, -122.39061 47.5...  2.388513e+07   
99     2004  MULTIPOLYGON (((-122.37238 47.58342, -122.3722...  7.024407e+07   
       2008  MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   
       2011  MULTIPOLYGON (((-122.37238 47.58342, -122.3722...  7.024407e+07   
       2014  MULTIPOLYGON (((-122.37238 47.58342, -122.3722...  7.024407e+07   
       2015  MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   
       2017  MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   
       2018  MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   

             index_right  SEGMENT_TY  SNDSEG_UPD  SHAPE_Leng  
NAME10 YEAR                                                   
95     2012        24124           6  2012-10-19  244.545395  
       2015        28568           6  2015-07-13   45.446456  
96     2004         3644           6  2004-05-19   80.834474  
       2011        19472           6  2011-02-24   86.871776  
       2017        19809           6  2017-07-19  130.463476  
97.01  2004         1267           6  2004-05-19  191.861908  
       2011        19578           6  2011-02-24  318.386061  
97.02  2004         1697           6  2004-05-19  164.874834  
       2015        28314           6  2015-07-24   82.784663  
99     2004         8776           6  2004-05-19  259.441455  
       2008        18126           6  2008-05-19   83.389786  
       2011        19435           6  2011-02-24  216.416233  
       2014        27617           6  2014-12-18   65.067733  
       2015        27855           6  2015-05-22   59.955099  
       2017        30139           6  2017-02-02  148.163251  
       2018        32190           6  2018-03-23  103.611745

In [20]:
#Automate this process into function
#Take multiindexed DF as shown above, convert and process to form below
lengths = pd.Series(43560*lanes_years['SHAPE_Leng'].values / lanes_years['SHAPE_Area'].values)

year_lane = []
tract_lane = []

for i in range(np.size(lengths)):
    year_lane.append(lanes_years.index.values[i][0])
for k in range(np.size(lengths)):
    tract_lane.append(lanes_years.index.values[k][1])

df_lanes = pd.DataFrame(np.stack((year_lane,tract_lane,lengths)).T)

In [21]:
df_lanes.head()

,0,1,2
0,1,2011,0.8714617619474354
1,10,2013,0.561771181757596
2,10,2017,0.8885033805558079
3,100.01,2017,0.3968428770965108
4,100.02,2004,0.5896069313812291


In [22]:
import os

In [23]:
print(os.path)

<module 'posixpath' from '/Users/stlp/opt/anaconda3/lib/python3.8/posixpath.py'>


In [25]:
#df_lanes.to_csv(r'~/Project/Prediction/df_lanes.csv')

In [26]:
#pd.read_csv('df_lanes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'df_lanes.csv'

In [ ]:
walk_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
walk = df_SND[walk_columns]
walk.head()

In [ ]:
walks_years = []
for k in walk['SNDSEG_UPD'].values:
    walks_years.append(int(k[0:4]))

walk_years = walks_years

In [ ]:
walk['YEAR'] = pd.Series(walk_years)

In [ ]:
walks = walk[walk['SEGMENT_TY']==5]

In [ ]:
walks_cleaned = walks.drop(columns=['SEGMENT_TY'])

In [ ]:
walks_data = walks_cleaned.dissolve('YEAR')
walks_data.head(n=25)

In [ ]:
walks_by_tract = gpd.sjoin(census_bounds_cleaned, walks_data, op='intersects')
walks_by_tract.dissolve(by=['index_right','NAME10'])
walks_by_tract.drop(columns=['geometry','SHAPE_Area','SNDSEG_UPD'])

In [ ]:
racks_df = gpd.read_file('Bike_Racks.geojson')
racks_df.head()

In [ ]:
park_times = []
for timestamp in street_signs['ADDDTTM'].values:
    x = timestamp[:10]
    x_time = datetime.datetime.fromtimestamp(int(x)).isoformat()
    x_year = (int(x_time[0:4]))
    park_times.append(x_year)

In [ ]:
street_signs['PARK YEAR'] = pd.Series(park_times)

In [ ]:
keep_list = ['PPP','PR','PRZ','PTIML','PINST','PCARPL']

In [ ]:
pop_url_2010 = 'https://gisrevprxy.seattle.gov/arcgis/rest/services/CENSUS_EXT/CENSUS_2010_BASICS/MapServer/15/query?where=1%3D1&outFields=SHAPE,GEOID10,NAME10,ACRES_TOTAL,Total_Population,OBJECTID&outSR=4326&f=json'
#pop_url_2000 =

In [ ]:
pop_2010 = gpd.read_file(pop_url_2010)

In [ ]:
pop_2010.head()

In [ ]:
test = pop_2010.NAME10.unique()

In [ ]:
test_sort = np.sort(test.astype(float))
test_sort

In [ ]:
pop_tracts = pd.merge(census_bounds, pop_2010, on = 'NAME10')

In [ ]:
pop_tracts.head()

In [ ]:
pop_tracts = pop_tracts[['NAME10', 'geometry_x', 'Total_Population']]

In [ ]:
pop_tracts.rename(columns={'geometry_x':'geometry'})

In [ ]:
total_pop = pop_tracts['Total_Population'].sum()
total_pop

In [ ]:
pop_tracts['Pop_fraction'] = pop_tracts['Total_Population']/total_pop
pop_tracts.head()

In [ ]:
years = list(range(2007, 2019))
years

In [ ]:
populations = [585436, 591870, 598539, 608660, 622694, 635928, 653588, 670109, 687386, 709631, 728661, 742235]

In [ ]:
pop_by_year = dict(zip(years, populations))
pop_by_year

In [ ]:
pop_by_year.get(2015)

In [ ]:
def est_tract_pop(year, pop_tracts, pop_by_year):
    pop_frac = pop_tracts['Pop_fraction'].values
    year_pop = pop_by_year.get(year)
    pop_tracts_year = pop_tracts
    pop_tracts_year['Total_Population'] = pop_frac*year_pop
    return pop_tracts_year

In [ ]:
pop_tracts_2007 = est_tract_pop(2007, pop_tracts, pop_by_year)
pop_tracts_2007.head()

In [ ]:
total_pop_2007 = pop_tracts_2007['Total_Population'].sum()
total_pop_2007

In [ ]:
df_lanes.head(n=50)
#Index 0 is NAME10, 1 is year, 2 is feet of bike lanes added in that region that year, adjusted for area of tract